# Homework #6

## GitHub Link: https://github.com/garcimat/EE-399/tree/main/HW6

### SHRED applied to SST dataset

This iPython notebook gives an introductory walkthrough to using SHRED models.  The dataset we consider is weekly mean sea-surface temperature as given by the NOAA Optimum Interpolation SST V2 dataset (https://psl.noaa.gov/data/gridded/data.noaa.oisst.v2.html).

SHRED (SHallow REcurrent Decoder) models are a network architecture that merges a recurrent layer (LSTM) with a shallow decoder network (SDN) to reconstruct high-dimensional spatio-temporal fields from a trajectory of sensor measurements of the field. More formally, the SHRED architecture can be written as 
$$ \mathcal {H} \left( \{ y_i \} _{i=t-k}^t \right) = \mathcal {F} \left( \mathcal {G} \left( \{ y_i \} _{i=t-k}^t \right) ; W_{RN}) ; W_{SD} \right)$$
where $\mathcal F$ is a feed forward network parameterized by weights $W_{SD}$, $\mathcal G$ is a LSTM network parameterized by weights $W_{RN}$, and $\{ y_i \} _{i=t-k}^t$ is a trajectory of sensor measurements of a high-dimensional spatio-temporal field $\{ x_i \} _{i=t-k}^t$.

We first randomly select 3 sensor locations and set the trajectory length (lags) to 52, corresponding to one year of measurements.

> Indented block



In [12]:
import numpy as np
from processdata import load_data
from processdata import TimeSeriesDataset
import models
import torch
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

num_sensors = 3 
lags = 52
load_X = load_data('SST')
n = load_X.shape[0]
m = load_X.shape[1]
sensor_locations = np.random.choice(m, size=num_sensors, replace=False)

We now select indices to divide the data into training, validation, and test sets.

In [13]:
train_indices = np.random.choice(n - lags, size=1000, replace=False)
mask = np.ones(n - lags)
mask[train_indices] = 0
valid_test_indices = np.arange(0, n - lags)[np.where(mask!=0)[0]]
valid_indices = valid_test_indices[::2]
test_indices = valid_test_indices[1::2]

sklearn's MinMaxScaler is used to preprocess the data for training and we generate input/output pairs for the training, validation, and test sets. 

In [14]:
sc = MinMaxScaler()
sc = sc.fit(load_X[train_indices])
transformed_X = sc.transform(load_X)

### Generate input sequences to a SHRED model
all_data_in = np.zeros((n - lags, lags, num_sensors))
for i in range(len(all_data_in)):
    all_data_in[i] = transformed_X[i:i+lags, sensor_locations]

### Generate training validation and test datasets both for reconstruction of states and forecasting sensors
device = 'cuda' if torch.cuda.is_available() else 'cpu'

train_data_in = torch.tensor(all_data_in[train_indices], dtype=torch.float32).to(device)
valid_data_in = torch.tensor(all_data_in[valid_indices], dtype=torch.float32).to(device)
test_data_in = torch.tensor(all_data_in[test_indices], dtype=torch.float32).to(device)

### -1 to have output be at the same time as final sensor measurements
train_data_out = torch.tensor(transformed_X[train_indices + lags - 1], dtype=torch.float32).to(device)
valid_data_out = torch.tensor(transformed_X[valid_indices + lags - 1], dtype=torch.float32).to(device)
test_data_out = torch.tensor(transformed_X[test_indices + lags - 1], dtype=torch.float32).to(device)

train_dataset = TimeSeriesDataset(train_data_in, train_data_out)
valid_dataset = TimeSeriesDataset(valid_data_in, valid_data_out)
test_dataset = TimeSeriesDataset(test_data_in, test_data_out)

We train the model using the training and validation datasets.

In [ ]:
shred = models.SHRED(num_sensors, m, hidden_size=64, hidden_layers=2, l1=350, l2=400, dropout=0.1).to(device)
validation_errors = models.fit(shred, train_dataset, valid_dataset, batch_size=64, num_epochs=1000, lr=1e-3, verbose=True, patience=5)

Training epoch 1
Error tensor(0.4812)
Training epoch 20
Error tensor(0.2260)
Training epoch 40
Error tensor(0.2176)
Training epoch 60
Error tensor(0.2156)
Training epoch 80
Error tensor(0.2147)
Training epoch 100
Error tensor(0.2061)
Training epoch 120
Error tensor(0.1978)


Finally, we generate reconstructions from the test set and print mean square error compared to the ground truth.

In [ ]:
test_recons = sc.inverse_transform(shred(test_dataset.X).detach().cpu().numpy())
test_ground_truth = sc.inverse_transform(test_dataset.Y.detach().cpu().numpy())
print(np.linalg.norm(test_recons - test_ground_truth) / np.linalg.norm(test_ground_truth))

## 1. Train the model and plot the results

In [ ]:
# Set the random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Set the number of sensors and lags
num_sensors = 3
lags = 52

# Load the data
load_X = load_data('SST')

# Get the dimensions of the data
n = load_X.shape[0]
m = load_X.shape[1]

# Randomly select sensor locations
sensor_locations = np.random.choice(m, size=num_sensors, replace=False)

# Split the data into training, validation, and test sets
train_indices = np.random.choice(n - lags, size=1000, replace=False)
mask = np.ones(n - lags)
mask[train_indices] = 0
valid_test_indices = np.arange(0, n - lags)[np.where(mask != 0)[0]]
valid_indices = valid_test_indices[::2]
test_indices = valid_test_indices[1::2]

# Preprocess the data using MinMaxScaler
sc = MinMaxScaler()
sc = sc.fit(load_X[train_indices])
transformed_X = sc.transform(load_X)

# Generate input sequences for the SHRED model
all_data_in = np.zeros((n - lags, lags, num_sensors))
for i in range(len(all_data_in)):
    all_data_in[i] = transformed_X[i:i + lags, sensor_locations]

# Convert the data to PyTorch tensors
device = 'cuda' if torch.cuda.is_available() else 'cpu'
train_data_in = torch.tensor(all_data_in[train_indices], dtype=torch.float32).to(device)
valid_data_in = torch.tensor(all_data_in[valid_indices], dtype=torch.float32).to(device)
test_data_in = torch.tensor(all_data_in[test_indices], dtype=torch.float32).to(device)

train_data_out = torch.tensor(transformed_X[train_indices + lags - 1], dtype=torch.float32).to(device)
valid_data_out = torch.tensor(transformed_X[valid_indices + lags - 1], dtype=torch.float32).to(device)
test_data_out = torch.tensor(transformed_X[test_indices + lags - 1], dtype=torch.float32).to(device)

# Create datasets for training, validation, and testing
train_dataset = TimeSeriesDataset(train_data_in, train_data_out)
valid_dataset = TimeSeriesDataset(valid_data_in, valid_data_out)
test_dataset = TimeSeriesDataset(test_data_in, test_data_out)

# Initialize the SHRED model
shred = models.SHRED(num_sensors, m, hidden_size=64, hidden_layers=2, l1=350, l2=400, dropout=0.1).to(device)

# Train the model
validation_errors = models.fit(shred, train_dataset, valid_dataset, batch_size=64, num_epochs=1000, lr=1e-3, verbose=True, patience=5)

# Plot the training and validation errors
plt.plot(validation_errors['train'], label='Train')
plt.plot(validation_errors['valid'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Mean Squared Error')
plt.title('Training and Validation Errors')
plt.legend()
plt.show()

# Generate reconstructions from the test set
test_recons = sc.inverse_transform(shred(test_dataset.X).detach().cpu().numpy())
test_ground_truth = sc.inverse_transform(test_dataset.Y.detach().cpu().numpy())

# Calculate the mean squared error
mse = np.linalg.norm(test_recons - test_ground_truth) / np.linalg.norm(test_ground_truth)
print("Mean Squared Error:", mse)

## 2. Analysis of the performance as a function of the time lag variable

In [ ]:
# Set the random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Set the number of sensors
num_sensors = 3

# Define a range of time lags to test
lags_range = [26, 39, 52, 65, 78]

# Initialize lists to store the mean squared errors for each lag
mse_values = []

# Load the data
load_X = load_data('SST')

# Get the dimensions of the data
n = load_X.shape[0]
m = load_X.shape[1]

# Randomly select sensor locations
sensor_locations = np.random.choice(m, size=num_sensors, replace=False)

for lags in lags_range:
    # Split the data into training, validation, and test sets
    train_indices = np.random.choice(n - lags, size=1000, replace=False)
    mask = np.ones(n - lags)
    mask[train_indices] = 0
    valid_test_indices = np.arange(0, n - lags)[np.where(mask != 0)[0]]
    valid_indices = valid_test_indices[::2]
    test_indices = valid_test_indices[1::2]

    # Preprocess the data using MinMaxScaler
    sc = MinMaxScaler()
    sc = sc.fit(load_X[train_indices])
    transformed_X = sc.transform(load_X)

    # Generate input sequences for the SHRED model
    all_data_in = np.zeros((n - lags, lags, num_sensors))
    for i in range(len(all_data_in)):
        all_data_in[i] = transformed_X[i:i + lags, sensor_locations]

    # Convert the data to PyTorch tensors
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    test_data_in = torch.tensor(all_data_in[test_indices], dtype=torch.float32).to(device)
    test_data_out = torch.tensor(transformed_X[test_indices + lags - 1], dtype=torch.float32).to(device)

    # Create a dataset for testing
    test_dataset = TimeSeriesDataset(test_data_in, test_data_out)

    # Initialize the SHRED model
    shred = models.SHRED(num_sensors, m, hidden_size=64, hidden_layers=2, l1=350, l2=400, dropout=0.1).to(device)

    # Load pre-trained weights (optional)
    # shred.load_state_dict(torch.load('shred_model.pt'))

    # Generate reconstructions from the test set
    test_recons = sc.inverse_transform(shred(test_dataset.X).detach().cpu().numpy())
    test_ground_truth = sc.inverse_transform(test_dataset.Y.detach().cpu().numpy())

    # Calculate the mean squared error
    mse = np.linalg.norm(test_recons - test_ground_truth) / np.linalg.norm(test_ground_truth)
    mse_values.append(mse)

# Plot the performance as a function of the time lag
plt.plot(lags_range, mse_values, marker='o')
plt.xlabel('Time Lag')
plt.ylabel('Mean Squared Error')
plt.title('Performance vs. Time Lag')
plt.show()

## 3. Analysis of the performance as a function of noise (add Gaussian noise to data)

In [ ]:
# Set the random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Set the number of sensors and lags
num_sensors = 3
lags = 52

# Define a range of noise levels to test
noise_levels = [0.0, 0.1, 0.2, 0.3, 0.4]

# Initialize lists to store the mean squared errors for each noise level
mse_values = []

# Load the data
load_X = load_data('SST')

# Get the dimensions of the data
n = load_X.shape[0]
m = load_X.shape[1]

# Randomly select sensor locations
sensor_locations = np.random.choice(m, size=num_sensors, replace=False)

for noise_level in noise_levels:
    # Add Gaussian noise to the data
    noisy_X = load_X + noise_level * np.random.randn(*load_X.shape)

    # Split the data into training, validation, and test sets
    train_indices = np.random.choice(n - lags, size=1000, replace=False)
    mask = np.ones(n - lags)
    mask[train_indices] = 0
    valid_test_indices = np.arange(0, n - lags)[np.where(mask != 0)[0]]
    valid_indices = valid_test_indices[::2]
    test_indices = valid_test_indices[1::2]

    # Preprocess the data using MinMaxScaler
    sc = MinMaxScaler()
    sc = sc.fit(noisy_X[train_indices])
    transformed_X = sc.transform(noisy_X)

    # Generate input sequences for the SHRED model
    all_data_in = np.zeros((n - lags, lags, num_sensors))
    for i in range(len(all_data_in)):
        all_data_in[i] = transformed_X[i:i + lags, sensor_locations]

    # Convert the data to PyTorch tensors
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    test_data_in = torch.tensor(all_data_in[test_indices], dtype=torch.float32).to(device)
    test_data_out = torch.tensor(transformed_X[test_indices + lags - 1], dtype=torch.float32).to(device)

    # Create a dataset for testing
    test_dataset = TimeSeriesDataset(test_data_in, test_data_out)

    # Initialize the SHRED model
    shred = models.SHRED(num_sensors, m, hidden_size=64, hidden_layers=2, l1=350, l2=400, dropout=0.1).to(device)

    # Load pre-trained weights (optional)
    # shred.load_state_dict(torch.load('shred_model.pt'))

    # Generate reconstructions from the test set
    test_recons = sc.inverse_transform(shred(test_dataset.X).detach().cpu().numpy())
    test_ground_truth = sc.inverse_transform(test_dataset.Y.detach().cpu().numpy())

    # Calculate the mean squared error
    mse = np.linalg.norm(test_recons - test_ground_truth) / np.linalg.norm(test_ground_truth)
    mse_values.append(mse)

# Plot the performance as a function of the noise level
plt.plot(noise_levels, mse_values, marker='o')
plt.xlabel('Noise Level')
plt.ylabel('Mean Squared Error')
plt.title('Performance vs. Noise Level')
plt.show()

## 4. Analysis of the performance as a function of the number of sensors

In [ ]:
# Set the random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Set the time lag and define a range of number of sensors to test
lags = 52
num_sensors_range = [1, 2, 3, 4, 5]

# Initialize lists to store the mean squared errors for each number of sensors
mse_values = []

# Load the data
load_X = load_data('SST')

# Get the dimensions of the data
n = load_X.shape[0]
m = load_X.shape[1]

for num_sensors in num_sensors_range:
    # Randomly select sensor locations
    sensor_locations = np.random.choice(m, size=num_sensors, replace=False)

    # Split the data into training, validation, and test sets
    train_indices = np.random.choice(n - lags, size=1000, replace=False)
    mask = np.ones(n - lags)
    mask[train_indices] = 0
    valid_test_indices = np.arange(0, n - lags)[np.where(mask != 0)[0]]
    valid_indices = valid_test_indices[::2]
    test_indices = valid_test_indices[1::2]

    # Preprocess the data using MinMaxScaler
    sc = MinMaxScaler()
    sc = sc.fit(load_X[train_indices])
    transformed_X = sc.transform(load_X)

    # Generate input sequences for the SHRED model
    all_data_in = np.zeros((n - lags, lags, num_sensors))
    for i in range(len(all_data_in)):
        all_data_in[i] = transformed_X[i:i + lags, sensor_locations]

    # Convert the data to PyTorch tensors
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    test_data_in = torch.tensor(all_data_in[test_indices], dtype=torch.float32).to(device)
    test_data_out = torch.tensor(transformed_X[test_indices + lags - 1], dtype=torch.float32).to(device)

    # Create a dataset for testing
    test_dataset = TimeSeriesDataset(test_data_in, test_data_out)

    # Initialize the SHRED model
    shred = models.SHRED(num_sensors, m, hidden_size=64, hidden_layers=2, l1=350, l2=400, dropout=0.1).to(device)

    # Generate reconstructions from the test set
    test_recons = sc.inverse_transform(shred(test_dataset.X).detach().cpu().numpy())
    test_ground_truth = sc.inverse_transform(test_dataset.Y.detach().cpu().numpy())

    # Calculate the mean squared error
    mse = np.linalg.norm(test_recons - test_ground_truth) / np.linalg.norm(test_ground_truth)
    mse_values.append(mse)

# Plot the performance as a function of the number of sensors
plt.plot(num_sensors_range, mse_values, marker='o')
plt.xlabel('Number of Sensors')
plt.ylabel('Mean Squared Error')
plt.title('Performance vs. Number of Sensors')
plt.show()